In [ ]:
import numpy as np
import scipy as sp
from numpy.polynomial import polynomial as poly
from numpy.polynomial import Polynomial
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

In [ ]:
degree = 5
poly_basis = []
for i in range(degree + 1):
    poly_basis.append(Polynomial.basis(deg= i))
    print(poly_basis[i])

In [ ]:
X = np.linspace(-np.pi, np.pi, 1000)
fig, ax = plt.subplots()
for f in poly_basis[1:]:
    sns.lineplot(
        x= X,
        y= f(X),
        ax= ax
    )

In [ ]:
orthonorm_basis = []
for i in range(degree + 1):
    new_basis = poly_basis[i]
    for j in range(i):
        numer = sp.integrate.quad((poly_basis[i] * orthonorm_basis[j]), -np.pi, np.pi)[0]
        denom = sp.integrate.quad((orthonorm_basis[j] ** 2), -np.pi, np.pi)[0]
        new_basis = new_basis -  ((numer / denom) * orthonorm_basis[j])
    orthonorm_basis.append(new_basis)
for i in range(len(orthonorm_basis)):
    norm = np.sqrt(np.reciprocal(sp.integrate.quad(orthonorm_basis[i] ** 2, -np.pi, np.pi)[0]))
    orthonorm_basis[i] = orthonorm_basis[i] * norm 

In [ ]:
for f in orthonorm_basis:
    print(f)

In [ ]:
fig, ax = plt.subplots()
for f in orthonorm_basis[1:]:
    sns.lineplot(
        x= X,
        y= f(X)
    )

In [ ]:
projections = []
for i in range(len(orthonorm_basis)):
    numer = sp.integrate.quad(lambda t: np.sin(t) * orthonorm_basis[i](t), -np.pi, np.pi)[0]
    denom = sp.integrate.quad(orthonorm_basis[i] ** 2, -np.pi, np.pi)[0]
    projections.append(
        numer * orthonorm_basis[i]
    )
    print(projections[i])
u_x = np.sum(projections)
u_x

In [ ]:
fig, ax = plt.subplots()
for f in projections:
    sns.lineplot(
        x= X,
        y= f(X)
    )

In [ ]:
domain = np.linspace(-np.pi, np.pi, 1000)
fit_poly, fit = Polynomial.fit(domain, np.sin(domain), deg= 5, full= True)
fit_poly

In [ ]:
Polynomial(fit_poly.convert().coef)

In [ ]:
for value, name in zip(fit, ["RSS", "rank", "singular values", "rcond"]):
    print(f"{name}: {value}")

In [ ]:
df = pd.DataFrame({
    "x": np.linspace(-np.pi, np.pi, 1000)
})
df["sin"] = np.sin(df["x"])
df["approx"] = u_x(df["x"])
df["polyfit"] = fit_poly(df["x"])

In [ ]:
np.sum((df["sin"] - df["polyfit"]) ** 2)

In [ ]:
np.sum((df["sin"] - df["approx"]) ** 2)

In [ ]:
df = pd.DataFrame({
    "x": np.linspace(-4, 4, 1000)
})
df["sin"] = np.sin(df["x"])
df["approx"] = u_x(df["x"])
df["polyfit"] = fit_poly(df["x"])

In [ ]:
fig, ax = plt.subplots(figsize= (10,10))
sns.lineplot(
    df.melt(id_vars= "x", value_vars= df.columns.drop("x"), value_name= "y", var_name= "function"),
    x= "x",
    y= "y",
    hue= "function",
    style= "function",
    linewidth= 3,
    alpha= 0.7
)
ax.vlines([-np.pi, np.pi], ymax= 1, ymin= -1, linestyles= "--", colors= "r")